In [1]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
with open('email_training_data.json') as file:
    training_data_json = json.load(file)
    df_train = pd.DataFrame(training_data_json)

with open('email_testing_data.json') as file:
    testing_data_json = json.load(file)
    df_test = pd.DataFrame(testing_data_json)

with open('emaildata.json') as file:
    data = json.load(file)
    df = pd.DataFrame(data)

In [3]:
df_train_y = df_train[['Category']].copy()
df_train_X = df_train[['Content']].copy()
df_test_y = df_test[['Category']].copy()
df_test_X = df_test[['Content']].copy()
df_train_y = np.squeeze(df_train_y)
df_test_y = np.squeeze(df_test_y)
df_train_X = np.squeeze(df_train_X)
df_test_X = np.squeeze(df_test_X)
df_train_y.shape, df_train_X.shape, df_test_y.shape, df_test_X.shape

((144,), (144,), (36,), (36,))

In [4]:
# from sklearn.model_selection import train_test_split

# df_train_X, df_test_X, df_train_y, df_test_y = train_test_split(
#     df['Content'], df['Category'], test_size=0.2)

In [5]:
# Use sklearn's CountVectorizer for bags-of-words approach

count_vectorizer = CountVectorizer(stop_words='english', max_features=1000, ngram_range=(1, 2))
X_train_count = count_vectorizer.fit_transform(df_train_X)
X_test_count = count_vectorizer.transform(df_test_X)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_count, df_train_y)
predictions = classifier.predict(X_test_count)

# Evaluate the model
accuracy = accuracy_score(df_test_y, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7222222222222222


In [6]:
# Use sklearn's TFIDVectorizer for bags-of-words approach

tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 1000, ngram_range = (1,2))
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train_X)
X_test_tfidf = tfidf_vectorizer.transform(df_test_X)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_tfidf, df_train_y)
predictions = classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(df_test_y, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6388888888888888


In [7]:
# Adding text length and word count as features in addition to CountVectorizer based model

X_train_count = X_train_count.toarray()
X_test_count = X_test_count.toarray()
X_train_count = np.concatenate([X_train_count, df_train_X.apply(len).values.reshape(-1, 1), df_train_X.apply(lambda x: len(x.split())).values.reshape(-1, 1)], axis=1)
X_test_count = np.concatenate([X_test_count, df_test_X.apply(len).values.reshape(-1, 1), df_test_X.apply(lambda x: len(x.split())).values.reshape(-1, 1)], axis=1)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_count, df_train_y)
predictions = classifier.predict(X_test_count)

# Evaluate the model
accuracy = accuracy_score(df_test_y, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.75


/Users/longjian/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
